# Test metadata extraction
Notebook to run the Brainplayback Task02 experiment for subject 05.

To recreate the environment run
`conda env create --name brainplayback --file environment.yml`

In [33]:
# Import requirements
import spotipy
import pandas

In [34]:
# subject name/id
subject_id = "sub-05"

In [35]:
# Setup Spotipy
scope = "user-library-read,user-read-playback-state,user-modify-playback-state"
sp = spotipy.Spotify(
    auth_manager=spotipy.oauth2.SpotifyOAuth(
        scope=scope,
        client_id="253a1345850d4a5c94995c1159d286e4",
        client_secret="9a03e1bcb8124eed9b661991e84bf477",
        redirect_uri="http://127.0.0.1:9090",
    ),
)

# choose deviceID (1st)
res = sp.devices()
devID = res["devices"][0]["id"]

## Import subject-specific selection of songs

In [36]:
# Start dataframe with three columns: Emotion, Name, Artist
df_meta = pandas.DataFrame(columns=["Emotion", "Name", "Artist"])

In [37]:
# Iterate on the music matrices (subjects, N=20)

for subject_id in [
    "sub-01",
    "sub-02",
    "sub-03",
    "sub-04",
    "sub-05",
    "sub-06",
    "sub-07",
    "sub-08",
    "sub-09",
    "sub-10",
    "sub-11",
    "sub-12",
    "sub-13",
    "sub-14",
    "sub-15",
    "sub-16",
    "sub-17",
    "sub-18",
    "sub-19",
    "sub-20",
]:
    # Print subject ID
    print(f"Processing {subject_id}")

    # read file
    D = pandas.read_table(f"../data/music_matrices/music_{subject_id}.tsv", index_col="Emotion")

    # extract list of emotions
    emotionList = D.index.values

    # Iterate through emotions
    for emotion in emotionList:
        # Iterate on the two songs
        for i in range(1, 3):
            # Get the track info based on the trackID
            song = D[f"trackID{str(i)}"][emotion]
            track_info = sp.track(song)

            # Concat the data to the dataframe
            df_meta = pandas.concat(
                [
                    df_meta,
                    pandas.DataFrame(
                        {
                            "Emotion": [emotion],
                            "Name": [track_info["name"]],
                            "Artist": [track_info["album"]["artists"][0]["name"]],
                        }
                    ),
                ],
                ignore_index=True,
            )

Processing sub-01
Processing sub-02
Processing sub-03
Processing sub-04
Processing sub-05
Processing sub-06
Processing sub-07
Processing sub-08
Processing sub-09
Processing sub-10
Processing sub-11
Processing sub-12
Processing sub-13
Processing sub-14
Processing sub-15
Processing sub-16
Processing sub-17
Processing sub-18
Processing sub-19
Processing sub-20


In [38]:
# remove duplicates
df_meta = df_meta.drop_duplicates()

# Sort by Emotion
df_meta = df_meta.sort_values(by="Emotion").reset_index(drop=True)

df_meta

,Emotion,Name,Artist
0,Joyful activation,Don't Stop Me Now - Remastered 2011,Queen
1,Joyful activation,Steal My Kisses,Ben Harper And The Innocent Criminals
2,Joyful activation,Never Gonna Give You Up,Rick Astley
3,Joyful activation,Gisela,Bárbara Tinoco
4,Joyful activation,Virtual Insanity,Various Artists
...,...,...,...
348,Wonder,If I Lose Myself - Alesso vs OneRepublic,Alesso
349,Wonder,Cantiga para quem sonha,Luiz Goes
350,Wonder,Walk Of Life,Dire Straits
351,Wonder,The Heart Asks Pleasure First,Michael Nyman


In [39]:
# export to tsv
df_meta.to_csv("../data/music_metadata.tsv", sep="\t", index=False)